<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Metrices/Descriptive_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation

Cloning into 'Objective-criterias-to-quantify-the-accuracy-of-explanation'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 220 (delta 46), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (220/220), 20.33 MiB | 4.79 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Updating files: 100% (13/13), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install shap
!pip install lime
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import pandas as pd
import numpy as np
import shap
from transformers import BertTokenizer
import scipy as sp
import torch
import lime
import tensorflow as tf
from lime.lime_text import LimeTextExplainer
import transformers
import torch.nn as nn

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)    

In [6]:
df = pd.read_csv('/content/Objective-criterias-to-quantify-the-accuracy-of-explanation/Dataset/test.csv')
df.head()

,hindi_text,label
0,मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा ह...,0
1,मैं किशोरावस्था से ही गंभीर अवसाद के दौर से गु...,1
2,मुझे शौक के तौर पर खाना बनाना पसंद है,0
3,"मैं इन दिनों बहुत कम महसूस कर रहा हूं, ऐसा महस...",1
4,हाल ही में ब्रिटेन की महारानी एलिजाबेथ का निधन...,2


In [7]:
text = df['hindi_text'][0]

In [8]:
test_data = df['hindi_text'][:50]
test_label = df['label'][:50]

In [9]:
# Define a function to preprocess the text data for the MuRIL model

def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()
      # predictions = tf.nn.softmax(logits)
      # prediction_label = class_name[np.argmax(predictions[0])]
  
  return scores


In [10]:
## generate explanation with LIME

class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

lime_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    label = test_label.iloc[i]
    explanation = explainer.explain_instance(text,adapter)
    exp = explanation.as_list()
    lime_explanations.append(exp)

In [11]:
## SHAP adaptor and predictor
def adapter_shap(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

def predictor(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    s = sp.special.logit(probas[:,1])
    return s

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor(i))
    return val

In [12]:
explainer = shap.Explainer(f_batch, tokenizer)
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer(test)
  values = shap_values[0].values
  return values


In [13]:
# Generate SHAP explanations for the test data
shap_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    exp = explain(text)
    shap_explanations.append(exp)

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

In [14]:
lime_scores = []
for instance in lime_explanations:
  individual_score = []
  for tuple in instance:
    feature, score = tuple
    individual_score.append(score)
  lime_scores.append(individual_score)

In [15]:
shap_scores = []
for instance in shap_explanations:
  individual_score = instance.tolist()
  shap_scores.append(individual_score)

In [109]:
def get_score_length(list):
  sum = 0
  for score in list:
    sum += len(score)
  return sum

In [111]:
def descriptive_accuracy_lime(lime_scores, threshold):
    """
    Computes the descriptive accuracy of LIME explanations given a threshold.
    """
    max_features = max([len(score) for score in lime_scores])
    num_instances = len(lime_scores)
    # Pad the SHAP scores with zeros to make them equal in length
    lime_scores_padded = []
    for score in lime_scores:
        if len(score) < max_features:
            score = np.pad(score, (0, max_features - len(score)), mode='constant')
        lime_scores_padded.append(score)
    lime_scores_padded = np.array(lime_scores_padded)
    score_list = []
    for instance in lime_scores_padded.tolist():
      score_list.append(np.array(instance))
    lime_correct = np.sum(np.absolute(score_list) >= threshold)
    lime_coverage = lime_correct / num_instances
    lime_acc = lime_correct / get_score_length(lime_scores)
    
    return lime_acc, lime_coverage

In [121]:
# Assuming that you have the LIME scores in a numpy array, lime_scores
threshold = 0.1
lime_acc, lime_coverage = descriptive_accuracy_lime(lime_scores, threshold)
print(f"LIME accuracy: {lime_acc*100:.3f}%, LIME coverage: {lime_coverage:.3f}")

LIME accuracy: 21.181%, LIME coverage: 2.080


In [117]:
def descriptive_accuracy_shap(shap_scores, threshold):
    """
    Computes the descriptive accuracy of LIME and SHAP explanations given a threshold.
    """
    max_features = max([len(score) for score in shap_scores])
    num_instances = len(shap_scores)
    # Pad the SHAP scores with zeros to make them equal in length
    shap_scores_padded = []
    for score in shap_scores:
        if len(score) < max_features:
            score = np.pad(score, (0, max_features - len(score)), mode='constant')
        shap_scores_padded.append(score)
    
    # Convert the padded SHAP scores to a numpy array
    shap_scores_padded = np.array(shap_scores_padded)
    shap_correct = np.sum(np.absolute(shap_scores_padded) >= threshold)
    shap_coverage = shap_correct / num_instances
    shap_acc = shap_correct / get_score_length(shap_scores)
  
    return shap_acc, shap_coverage

In [122]:
# Assuming that you have the SHAP scores in a numpy array, lime_scores
threshold = 0.1
shap_acc, shap_coverage = descriptive_accuracy_shap(shap_explanations,threshold)
print(f"SHAP accuracy: {shap_acc*100:.3f}%, SHAP coverage: {shap_coverage:.3f}")

SHAP accuracy: 43.324%, SHAP coverage: 6.360
